# Testing chatbot memory with Langchain



#### Select the baseline model

In [ ]:
!pip install langchain_community langchain openai tiktoken ollama OllamaEmbeddings

ERROR: Could not find a version that satisfies the requirement OllamaEmbeddings (from versions: none)
ERROR: No matching distribution found for OllamaEmbeddings


In [ ]:
!pip install ollama

  Using cached ollama-0.4.8-py3-none-any.whl.metadata (4.7 kB)


In [ ]:
!apt install pciutils -y
!curl -fsSL https://ollama.com/install.sh | sh
!ollama run llama3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (1,140 kB/s)
Selecting previously unselected package pci.ids.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1ubuntu0.1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Selecting previously unse

In [ ]:
!pip install langchain_community

  Using cached langchain_community-0.3.22-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


In [ ]:
!ollama pull llama3

[GIN] 2025/04/27 - 15:21:27 | 200 |      71.736µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ time=2025-04-27T15:21:28.155Z level=INFO source=download.go:177 msg="downloading 6a0746a1ec1a in 16 291 MB part(s)"
pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling 6a0746a1ec1a:   0% ▕▏ 673 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   0% ▕▏  10 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   1% ▕▏  50 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   2% ▕▏  94 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   2% ▕▏ 116 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   3% ▕▏ 141 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   3% ▕▏ 156 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   4% ▕▏ 186 MB/4.7 GB                  pulling manifest 

In [ ]:
MODEL = "llama3" # https://ollama.com/library/llama3

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Give me an inspirational quote")

<ipython-input-13-6f758df09079>:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
<ipython-input-13-6f758df09079>:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


time=2025-04-27T15:24:50.467Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:24:50.468Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
time=2025-04-27T15:24:50.493Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:24:50.520Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:24:50.521Z level=INFO source=server.go:105 msg="system memory" total="12.7 GiB" free="11.6 GiB" free_swap="0 B"
time=2025-04-27T15:24:50.521Z level=WARN source=ggml.go:152 msg="key not found" key=llama.vision.block_count default=0
time=2025-04-27T15:24:50.521Z level=WARN source=ggml.go:152 msg="key not found" key=llama.attention.key_length default=128
time=2025-04-27T15:24:50.521Z level=WARN source=ggml.go:152 msg="key not found" key=llama.attention.value_length default=128
time=2025-04-27T15:24:50.522Z level=INFO source=server.go:1

'Here\'s one:\n\n"Believe you can and you\'re halfway there." - Theodore Roosevelt\n\nI hope it inspires you to tackle your challenges and pursue your goals with confidence!'

In [ ]:
from langchain.callbacks import get_openai_callback
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

## Type 1: ConversationBufferMemory

Retains previous iterations

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory


conversation_buf = ConversationChain(llm=model,verbose=True, memory=ConversationBufferMemory())

count_tokens(conversation_buf,"Hi, i am not able to start my dell laptop")

<ipython-input-16-0024b1652ece>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  conversation_buf = ConversationChain(llm=model,verbose=True, memory=ConversationBufferMemory())
<ipython-input-16-0024b1652ece>:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_buf = ConversationChain(llm=model,verbose=True, memory=ConversationBufferMemory())
<ipython-input-15-573a7f815b36>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(query)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, i am not able to start my dell laptop
AI:
time=2025-04-27T15:27:39.195Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:27:39.195Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 15:29:58 | 200 |         2m19s |       127.0.0.1 | POST     "/api/generate"
> Finished chain.
Spent a total of 0 tokens



"Sorry to hear that you're having trouble starting your Dell laptop! Can you tell me more about what's happening? Are you getting any error messages or is the screen just blank? Have you tried pressing the power button for a few seconds to see if it boots up eventually?\n\nAlso, have you recently installed any new software or drivers on your laptop that might be causing the issue? And are you using any external peripherals like a USB keyboard or mouse that could potentially interfere with the boot process?\n\nI'm happy to help you troubleshoot this problem!"

In [ ]:
count_tokens(conversation_buf,"It is a dell xps 17")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, i am not able to start my dell laptop
AI: Sorry to hear that you're having trouble starting your Dell laptop! Can you tell me more about what's happening? Are you getting any error messages or is the screen just blank? Have you tried pressing the power button for a few seconds to see if it boots up eventually?

Also, have you recently installed any new software or drivers on your laptop that might be causing the issue? And are you using any external peripherals like a USB keyboard or mouse that could potentially interfere with the boot process?

I'm happy to help you troubleshoot this problem!
Human: It is a dell xps 17
AI:
time=2025-04-27T15:30:57.469Z leve

"A Dell XPS 17! That's a powerful laptop. Okay, let me dig deeper.\n\nAs you've mentioned your Dell XPS 17 isn't starting up, have you tried booting it in safe mode or using the F2 key to enter the BIOS settings? Sometimes, the issue could be related to a third-party driver or software that's causing conflicts during startup.\n\nAdditionally, I'd like to know if you've recently made any changes to your laptop's configuration, such as updating the BIOS or installing new firmware. It's also possible that there might be an issue with the power cord or outlet you're using to charge the laptop.\n\nCan you tell me what happens when you press the power button? Does it just not turn on at all, or does it make any attempts to boot up before failing?\n\nLastly, have you checked the battery level of your XPS 17? If the battery is extremely low, it might prevent the laptop from turning on."

In [ ]:
print(conversation_buf.memory.buffer)

Human: Hi, i am not able to start my dell laptop
AI: Sorry to hear that you're having trouble starting your Dell laptop! Can you tell me more about what's happening? Are you getting any error messages or is the screen just blank? Have you tried pressing the power button for a few seconds to see if it boots up eventually?

Also, have you recently installed any new software or drivers on your laptop that might be causing the issue? And are you using any external peripherals like a USB keyboard or mouse that could potentially interfere with the boot process?

I'm happy to help you troubleshoot this problem!
Human: It is a dell xps 17
AI: A Dell XPS 17! That's a powerful laptop. Okay, let me dig deeper.

As you've mentioned your Dell XPS 17 isn't starting up, have you tried booting it in safe mode or using the F2 key to enter the BIOS settings? Sometimes, the issue could be related to a third-party driver or software that's causing conflicts during startup.

Additionally, I'd like to know 

## Type 2: ConversationSummaryMemory

Retains previous iterations summary. It asks the model for a summary of the input and them stores that.

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=model)
conversation_sum = ConversationChain(llm=model, memory=memory,verbose=True)


<ipython-input-19-9113e2261363>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=model)


In [ ]:
count_tokens(conversation_sum,"Hi, i am not able to start my dell laptop")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, i am not able to start my dell laptop
AI:
time=2025-04-27T15:36:58.472Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:36:58.473Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 15:38:34 | 200 |         1m35s |       127.0.0.1 | POST     "/api/generate"
time=2025-04-27T15:38:34.072Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:38:34.073Z level=WARN source=types.go:570 msg="invalid option provided" option=tfs_z
[GIN] 2025/04/27 - 15:42:02 | 200 |         3m28s |       127.0.0.1 | POST     "/api/generate"

"Sorry to hear that your Dell laptop won't turn on! Can you tell me more about what's happening? Is it not turning on at all, or is it just freezing or crashing during startup? And have you tried any troubleshooting steps so far?\n\nBy the way, I'm an AI assistant trained by a team of experts in natural language processing and computer science. My training data includes a vast amount of information about various technologies, including laptops like yours!"

In [ ]:
count_tokens(conversation_sum,"It is a dell xps 17")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Current summary:
None (initial conversation)

New lines of conversation:

Human: Hi, i am not able to start my dell laptop
AI: Sorry to hear that your Dell laptop won't turn on! Can you tell me more about what's happening? Is it not turning on at all, or is it just freezing or crashing during startup? And have you tried any troubleshooting steps so far?

New summary:
A human is unable to start their Dell laptop and has reached out for assistance.
Human: It is a dell xps 17
AI:
time=2025-04-27T15:44:57.666Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-04-27T15:44:57.667Z level=WARN source=types.go:570 msg="invalid option p

KeyboardInterrupt: 

In [ ]:
count_tokens(conversation_sum,"How do i start it up")

## Type 3: ConversationBufferWindowMemory


Remembers k previous interations

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
conversation_bufw = ConversationChain(llm=model, memory=ConversationBufferWindowMemory(k=1),verbose=False)

In [ ]:
count_tokens(conversation_bufw,"Hi, i am not able to start my dell laptop")

In [ ]:
count_tokens(conversation_bufw,"It is a dell xps 17")

In [ ]:
count_tokens(conversation_bufw,"How do i start it up")

## Type 4: ConversationSummaryBufferMemory

Combination of ConversationSummaryMemory and ConversationBufferWindowMemory


In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
conversation_sumbuf = ConversationChain(llm=model, memory=ConversationSummaryBufferMemory(llm=model),verbose=True)


In [ ]:
count_tokens(conversation_sumbuf,"Hi, i am not able to start my dell laptop")

In [ ]:
count_tokens(conversation_sumbuf,"It is a dell xps 17")

In [ ]:
count_tokens(conversation_sumbuf,"How do i start it up")